In [1]:
from simulacion import Simulacion
from estrategia import Estrategia
import parametros_simulacion as ps
import random
from threading import Thread
from funciones import Archivos as ar
random.seed(ps.SEED)
from copy import deepcopy
import scipy.stats as st
import numpy as np

In [15]:
class Analisis:
    def __init__(self, resultados_estrategias, cambio_politica):
        self.resultados_estrategias = resultados_estrategias
        self.cambio_politica = cambio_politica
        self.intervalo_confianza_derivaciones = {}
        self.intervalo_confianza_espera_WL = {}
        self.intervalo_confianza_costos_muertos = {}
        self.intervalo_confianza_costos_diarios = {}
        self.intervalo_de_confianza()
    
    def intervalo_de_confianza(self):
        costos_diarios = {}
        costos_WL = {}
        costos_derivaciones = {}
        costos_muertos = {}
        for key in self.resultados_estrategias["Funcion objetivo"].keys():
            costos_diarios[key] = list(self.resultados_estrategias["Funcion objetivo"][key].values())
        for key in self.resultados_estrategias["Costos muertos WL"].keys():
            costos_WL[key] = list(self.resultados_estrategias["Costos muertos WL"][key].values())
        for key in self.resultados_estrategias["Costos derivaciones"].keys():
            costos_derivaciones[key] = list(self.resultados_estrategias["Costos derivaciones"][key].values())
        for key in self.resultados_estrategias["Costos muertos hospitales"].keys():
            costos_muertos[key] = list(self.resultados_estrategias["Costos muertos hospitales"][key].values())
        

        
        for key in self.resultados_estrategias["Funcion objetivo"].keys():
            self.intervalo_confianza_espera_WL[key] = st.norm.interval(confidence = 0.95, loc = np.mean(costos_WL[key]), scale = st.sem(costos_WL[key]))
            self.intervalo_confianza_costos_muertos[key] = st.norm.interval(confidence = 0.95, loc = np.mean(costos_muertos[key]), scale = st.sem(costos_muertos[key]))
            self.intervalo_confianza_derivaciones[key] = st.norm.interval(confidence = 0.95, loc = np.mean(costos_derivaciones[key]), scale = st.sem(costos_derivaciones[key]))
            self.intervalo_confianza_costos_diarios[key] = st.norm.interval(confidence = 0.95, loc = np.mean(costos_diarios[key]), scale = st.sem(costos_diarios[key]))


    def chequear_intervalo(self):
        valores_WL_inicial = list(self.estrategia_inicial.costos_espera_WL_simulacion.values())
        valores_costos_muertos_inicial = list(self.estrategia_inicial.costos_muertos_hospitales_simulacion.values())
        valores_derivaciones_inicial = list(self.estrategia_inicial.costos_derivacion_simulacion.values())

        
        for valor in valores_WL_inicial:
            if self.intervalo_confianza_espera_WL[0] > valor or self.intervalo_confianza_espera_WL[1] < valor:
                self.condicion_WL = False

        for valor in valores_costos_muertos_inicial:
            if self.intervalo_confianza_costos_muertos[0] > valor or self.intervalo_confianza_costos_muertos[1] < valor:
                self.condicion_costos_muertos = False

        for valor in valores_derivaciones_inicial:
            if self.intervalo_confianza_derivaciones[0] > valor or self.intervalo_confianza_derivaciones[1] < valor:
                self.condicion_derivaciones = False




In [16]:
resultados_estra = ar('None').leer_resultados_estrategias()
analisis = Analisis(resultados_estra, 4)

In [20]:
analisis.intervalo_confianza_costos_muertos

{'Estrategia 1': (787.4565072876396, 829.1055536672816),
 'Estrategia 2': (-0.5194025724901523, 2.0502359020359675),
 'Estrategia 3': (-0.027016951500327117, 0.0833043788109917),
 'Estrategia 4': (-0.0016575105460767126, 0.14237607882273817),
 'Estrategia 5': (-0.005069974196193226, 0.08950111516219009),
 'Estrategia 6': (0.0006575545958250051, 0.1286029131945565)}